In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import Word2Vec,KeyedVectors
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,LancasterStemmer
import string
import re
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("DataNeuron_Text_Similarity.csv")
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [3]:
new_df = df.copy()

In [4]:
def remove_punc(df):
    pattern = r'[' + string.punctuation + ']'
    df['text1']=df['text1'].map(lambda m:re.sub(pattern," ",m))
    df['text2']=df['text2'].map(lambda m:re.sub(pattern," ",m))
    return df

remove_punc(df)
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3 2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [5]:
def remove_digits(df):
    df['text1'] = df['text1'].apply(lambda row: re.sub('\d.','',row))
    df['text1'] = df['text1'].apply(lambda row: re.sub('\d.','',row))
    return df

remove_digits(df)

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak cotonsport hearts of oak set up ...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...
...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...
2996,uto play at grammy awards show irish rock band...,israel looks to us for bank chief israel has a...
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...
2998,belle named best scottish band belle sebas...,mido makes third apology ahmed mido hossam h...


In [6]:
def lower(df):
    df['text1'] = df['text1'].map(lambda row: row.lower())
    df['text2'] = df['text2'].map(lambda row: row.lower())
    return df

lower(df)

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak cotonsport hearts of oak set up ...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...
...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...
2996,uto play at grammy awards show irish rock band...,israel looks to us for bank chief israel has a...
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...
2998,belle named best scottish band belle sebas...,mido makes third apology ahmed mido hossam h...


In [7]:
from nltk.tokenize import word_tokenize
def tokenize(df):
    df['text1'] = df['text1'].apply(word_tokenize)
    df['text2'] = df['text2'].apply(word_tokenize)
    return df

tokenize(df)

,text1,text2
0,"[broadband, challenges, tv, viewing, the, numb...","[gardener, wins, double, in, glasgow, britain,..."
1,"[rap, boss, arrested, over, drug, find, rap, m...","[amnesty, chief, laments, war, failure, the, l..."
2,"[player, burn, out, worries, robinson, england...","[hanks, greeted, at, wintry, premiere, hollywo..."
3,"[hearts, of, oak, cotonsport, hearts, of, oak,...","[redford, s, vision, of, sundance, despite, sp..."
4,"[sir, paul, rocks, super, bowl, crowds, sir, p...","[mauresmo, opens, with, victory, in, la, ameli..."
...,...,...
2995,"[uk, directors, guild, nominees, named, martin...","[steel, firm, to, cut, 45, 000, jobs, mittal, ..."
2996,"[uto, play, at, grammy, awards, show, irish, r...","[israel, looks, to, us, for, bank, chief, isra..."
2997,"[pountney, handed, ban, and, fine, northampton...","[india, and, iran, in, gas, export, deal, indi..."
2998,"[belle, named, best, scottish, band, belle, se...","[mido, makes, third, apology, ahmed, mido, hos..."


In [8]:

#on tokenized data
def remove_stopwords(df):
    stopword_list = stopwords.words("english")
    stopword_list.remove("no")
    stopword_list.remove("nor")
    stopword_list.remove("not")
    df['text1'] = df['text1'].apply(lambda x: [item for item in x if item not in stopword_list])
    df['text2'] = df['text2'].apply(lambda x: [item for item in x if item not in stopword_list])
    return df

remove_stopwords(df)

,text1,text2
0,"[broadband, challenges, tv, viewing, number, e...","[gardener, wins, double, glasgow, britain, jas..."
1,"[rap, boss, arrested, drug, find, rap, mogul, ...","[amnesty, chief, laments, war, failure, lack, ..."
2,"[player, burn, worries, robinson, england, coa...","[hanks, greeted, wintry, premiere, hollywood, ..."
3,"[hearts, oak, cotonsport, hearts, oak, set, gh...","[redford, vision, sundance, despite, sporting,..."
4,"[sir, paul, rocks, super, bowl, crowds, sir, p...","[mauresmo, opens, victory, la, amelie, mauresm..."
...,...,...
2995,"[uk, directors, guild, nominees, named, martin...","[steel, firm, cut, 45, 000, jobs, mittal, stee..."
2996,"[uto, play, grammy, awards, show, irish, rock,...","[israel, looks, us, bank, chief, israel, asked..."
2997,"[pountney, handed, ban, fine, northampton, coa...","[india, iran, gas, export, deal, india, signed..."
2998,"[belle, named, best, scottish, band, belle, se...","[mido, makes, third, apology, ahmed, mido, hos..."


In [9]:
def lemetize(df):
    lm = WordNetLemmatizer()
    df['text1']=df['text1'].apply(lambda row: [lm.lemmatize(word) for word in row])
    df['text2']=df['text2'].apply(lambda row: [lm.lemmatize(word) for word in row])
    return df

lemetize(df)

,text1,text2
0,"[broadband, challenge, tv, viewing, number, eu...","[gardener, win, double, glasgow, britain, jaso..."
1,"[rap, bos, arrested, drug, find, rap, mogul, m...","[amnesty, chief, lament, war, failure, lack, p..."
2,"[player, burn, worry, robinson, england, coach...","[hank, greeted, wintry, premiere, hollywood, s..."
3,"[heart, oak, cotonsport, heart, oak, set, ghan...","[redford, vision, sundance, despite, sporting,..."
4,"[sir, paul, rock, super, bowl, crowd, sir, pau...","[mauresmo, open, victory, la, amelie, mauresmo..."
...,...,...
2995,"[uk, director, guild, nominee, named, martin, ...","[steel, firm, cut, 45, 000, job, mittal, steel..."
2996,"[uto, play, grammy, award, show, irish, rock, ...","[israel, look, u, bank, chief, israel, asked, ..."
2997,"[pountney, handed, ban, fine, northampton, coa...","[india, iran, gas, export, deal, india, signed..."
2998,"[belle, named, best, scottish, band, belle, se...","[mido, make, third, apology, ahmed, mido, hoss..."


In [10]:
def detokenize(df):
    df['text1']=df['text1'].apply(lambda row: " ".join(row) )
    df['text2']=df['text2'].apply(lambda row: " ".join(row) ) 
    return df
    
detokenize(df)

,text1,text2
0,broadband challenge tv viewing number european...,gardener win double glasgow britain jason gard...
1,rap bos arrested drug find rap mogul marion su...,amnesty chief lament war failure lack public o...
2,player burn worry robinson england coach andy ...,hank greeted wintry premiere hollywood star to...
3,heart oak cotonsport heart oak set ghanaian co...,redford vision sundance despite sporting cordu...
4,sir paul rock super bowl crowd sir paul mccart...,mauresmo open victory la amelie mauresmo maria...
...,...,...
2995,uk director guild nominee named martin scorses...,steel firm cut 45 000 job mittal steel one wor...
2996,uto play grammy award show irish rock band uar...,israel look u bank chief israel asked u banker...
2997,pountney handed ban fine northampton coach bud...,india iran gas export deal india signed 40bn £...
2998,belle named best scottish band belle sebastian...,mido make third apology ahmed mido hossam made...


In [11]:
vocab= []
for text in df["text1"]:
    vocab.append(text.split(" "))
    
        
print("Text1 column completed")
        
for text in df["text2"]:
    vocab.append(text.split(" "))

Text1 column completed


In [12]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    vector_size=200
)

In [13]:
model.build_vocab(vocab)

In [14]:
model.train(vocab, total_examples=model.corpus_count, epochs=model.epochs)

(6649193, 6836615)

In [15]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc] , axis =0)

In [16]:
len(model.wv.index_to_key)

21879

In [17]:
X=[]
Y =[]
for doc in tqdm(df["text1"].values):
    X.append(document_vector(doc))

for doc in tqdm(df["text2"].values):
    Y.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [01:21<00:00, 36.99it/s]


In [18]:


sim_score =[]
for i in tqdm(range(len(X))):
    Xi = X[i].reshape(1, -1)
    Yi = Y[i].reshape(1, -1)
    s = cosine_similarity(Xi ,Yi)
    sim_score.append(s)

100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 1736.46it/s]


In [27]:
sim_score[0].shape

(1, 1)

In [10]:
sim_score = [round(s,2) for s in sim_score]
df["similarity_score"] = (sim_score)
df.head()

NameError: name 'sim_score' is not defined

In [30]:
df.to_csv("texts_with_similarity_score.csv")

In [33]:
def preprocess(text):
    # remove punctuation:
    
    pattern = r'[' + string.punctuation + ']'
    
    text = re.sub(pattern,'',text)
    
    # remove digits:
    
    text = re.sub('\d.','',text)
    
    # lower
    
    text = text.lower()
    
    # tokenize
    
    text = word_tokenize(text)
    
    # remove stopwords:
    
    stopword_list = stopwords.words("english")
    stopword_list.remove("no")
    stopword_list.remove("nor")
    stopword_list.remove("not")
    text = [word for word in text if word not in stopword_list]
    
    # lemetize
    
    lm = WordNetLemmatizer()
    text =[ lm.lemmatize(word) for word in text]
    
    return " ".join(text)


In [27]:
import pickle
pickle.dump(model , open("model.pkl" , "wb"))

In [32]:
def document_vector(text):
    doc = [word for word in text.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc] , axis =0)

In [35]:
def similarity(text1 , text2):
    x = document_vector(preprocess(text1)).reshape(1, -1)
    y = document_vector(preprocess(text2)).reshape(1, -1)
    s = cosine_similarity(x,y)
    return s

In [29]:
text1 =df["text1"][0]
text2 = df["text2"][0]

In [40]:
similarity(text1, text2)[0][0]

0.2896134